# Carregando e preparando os dados

In [3]:
import pandas as pd

dados = pd.read_excel("dados/dados_escores.xlsx")
dados.head()

,Ponto,PontoID,Data,Campo,Equipe,Especie01,Avaliador,Altura,EstDesenv,Invasoras,Cupins,CobertSolo,DispForr,DispFolhVerd,CondAtual,PotProd,Degrad,Manejo
0,M1001-E1-P1,750,2022-03-20,2,1,Brachiaria brizantha,Maria Eduarda Passos,2024-04-17 00:00:00,2024-05-03 00:00:00,4.0,2024-05-02 00:00:00,5.0,2024-05-03 00:00:00,3.0,2024-05-02 00:00:00,3.0,2024-05-03 00:00:00,2024-05-02 00:00:00
1,M1001-E1-P1,750,2022-03-20,2,1,Brachiaria brizantha,João Abrão,2024-04-17 00:00:00,2024-05-03 00:00:00,2024-05-04 00:00:00,3.0,2024-05-04 00:00:00,2024-05-03 00:00:00,2024-05-03 00:00:00,3.0,2024-05-03 00:00:00,2024-05-03 00:00:00,2024-05-02 00:00:00
2,M1001-E1-P1,750,2022-03-20,2,1,Brachiaria brizantha,Lara Guilarducci,2024-04-17 00:00:00,2024-05-03 00:00:00,2024-05-04 00:00:00,2024-05-03 00:00:00,2024-05-05 00:00:00,3.0,2024-05-02 00:00:00,3.0,2024-05-03 00:00:00,3.0,3.0
3,M1001-E1-P1,750,2022-03-20,2,1,Brachiaria brizantha,Victor,2024-04-17 00:00:00,2024-05-03 00:00:00,2024-05-03 00:00:00,3.0,5.0,3.0,3.0,3.0,3.0,2024-05-03 00:00:00,2024-05-02 00:00:00
4,M1001-E1-P3,748,2022-03-20,2,1,Brachiaria brizantha,João Abrão,23.0,5.0,2.0,2.0,4.0,3.0,3.0,2024-05-03 00:00:00,4.0,2024-05-02 00:00:00,2024-05-03 00:00:00


Estranhamente algumas colunas apresentaram tanto valores float quando valores de datetime. Isso não era para acontecer... Além disso, existe um padrão: o dia representa o valor à esquerda do "." do valor float, enquanto o mês representa o valor à direita do "." do valor float, de modo que facilmente podemos consertar esse problema.

Vamos filtrar as colunas que precisamos consertar:

In [4]:
colunas_para_consertar = [col for col in dados.columns if col not in ["Ponto", "PontoID", "Data", "Campo", "Equipe",
                                                                     "Especie01", "Avaliador"]]

print("Estas são as colunas para consertar:")
print(colunas_para_consertar)

Estas são as colunas para consertar:
['Altura', 'EstDesenv', 'Invasoras', 'Cupins', 'CobertSolo', 'DispForr', 'DispFolhVerd', 'CondAtual', 'PotProd', 'Degrad', 'Manejo']


Agora vamos criar e aplicar uma função para "transformar" os valores datetime em seus valores correspondentes em float:

In [15]:
# função para corrigir valores contínuos que foram codificados como datas
def corrigir_valores(valor):
    # se o valor for uma string, mantenha como está
    if type(valor) == str or type(valor) == float:
        return valor
    # se for data, conserte
    else:
        return str(valor.day) + "." + str(valor.month)

# consertando todas as colunas necessárias
for col in colunas_para_consertar:
    dados[col] = dados[col].apply(corrigir_valores)

# mostrando primeiros valores para avaliarmos se deu certo
dados.head(15)

,Ponto,PontoID,Data,Campo,Equipe,Especie01,Avaliador,Altura,EstDesenv,Invasoras,Cupins,CobertSolo,DispForr,DispFolhVerd,CondAtual,PotProd,Degrad,Manejo
0,M1001-E1-P1,750,2022-03-20,2,1,Brachiaria brizantha,Maria Eduarda Passos,17.4,3.5,4.0,2.5,5.0,3.5,3.0,2.5,3.0,3.5,2.5
1,M1001-E1-P1,750,2022-03-20,2,1,Brachiaria brizantha,João Abrão,17.4,3.5,4.5,3.0,4.5,3.5,3.5,3.0,3.5,3.5,2.5
2,M1001-E1-P1,750,2022-03-20,2,1,Brachiaria brizantha,Lara Guilarducci,17.4,3.5,4.5,3.5,5.5,3.0,2.5,3.0,3.5,3.0,3.0
3,M1001-E1-P1,750,2022-03-20,2,1,Brachiaria brizantha,Victor,17.4,3.5,3.5,3.0,5.0,3.0,3.0,3.0,3.0,3.5,2.5
4,M1001-E1-P3,748,2022-03-20,2,1,Brachiaria brizantha,João Abrão,23.0,5.0,2.0,2.0,4.0,3.0,3.0,3.5,4.0,2.5,3.5
5,M1001-E1-P3,748,2022-03-20,2,1,Brachiaria brizantha,Lara Guilarducci,23.0,5.0,2.0,2.5,5.0,3.5,3.0,3.5,4.0,2.0,4.5
6,M1001-E1-P3,748,2022-03-20,2,1,Brachiaria brizantha,Victor,23.0,4.5,2.0,2.0,4.5,3.0,2.5,3.0,4.0,1.5,4.0
7,M1001-E1-P3,748,2022-03-20,2,1,Brachiaria brizantha,Maria Eduarda Passos,23.0,5.0,2.5,2.0,4.5,3.0,3.0,3.5,4.0,1.5,4.0
8,M1009-E3-P1,713,2022-03-20,2,3,Brachiaria brizantha,Maria Eduarda Monteiro,11.2,3.5,6.0,1.5,3.5,2.5,2.0,2.0,2.0,5.5,2.0
9,M1009-E3-P1,713,2022-03-20,2,3,Brachiaria brizantha,Sabrina Beda,11.2,2.5,6.0,2.0,2.0,2.5,2.0,2.5,2.0,6.0,2.0


As colunas, no entanto, continuam sendo tratadas como strings. Vamos convertê-las para float.

In [16]:
# convertendo colunas que estão codificadas como string para float
for col in colunas_para_consertar:
    dados[col] = dados[col].astype(float)

dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Ponto         1599 non-null   object        
 1   PontoID       1599 non-null   object        
 2   Data          1599 non-null   datetime64[ns]
 3   Campo         1599 non-null   int64         
 4   Equipe        1599 non-null   int64         
 5   Especie01     1587 non-null   object        
 6   Avaliador     1599 non-null   object        
 7   Altura        1565 non-null   float64       
 8   EstDesenv     1580 non-null   float64       
 9   Invasoras     1597 non-null   float64       
 10  Cupins        1593 non-null   float64       
 11  CobertSolo    1593 non-null   float64       
 12  DispForr      1586 non-null   float64       
 13  DispFolhVerd  1586 non-null   float64       
 14  CondAtual     1586 non-null   float64       
 15  PotProd       1586 non-null   float64 

Pela análise acima, vemos que existem valores faltantes. Vamos visualizar as linhas com valores faltantes:

In [18]:
# visualizando linhas com valores faltantes
dados[dados.isna().any(axis=1)]

,Ponto,PontoID,Data,Campo,Equipe,Especie01,Avaliador,Altura,EstDesenv,Invasoras,Cupins,CobertSolo,DispForr,DispFolhVerd,CondAtual,PotProd,Degrad,Manejo
211,M1512-E5-P4,617,2022-03-19,2,5,Brachiaria brizantha,Ikanuza Pimentel,NaN,NaN,7.0,1.0,6.0,1.0,1.0,1.0,1.0,7.0,1.0
212,M1512-E5-P4,617,2022-03-19,2,5,Brachiaria brizantha,Lara Guilarducci,NaN,NaN,7.0,1.5,6.5,1.0,1.0,1.0,1.0,7.0,1.0
213,M1512-E5-P4,617,2022-03-19,2,5,Brachiaria brizantha,Wilton Ladeira,NaN,NaN,7.0,1.5,6.5,1.0,1.0,1.0,1.0,7.0,1.0
341,M1750-E5-P55,55,2022-03-31,3,5,NaN,Carlos Emanuell,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,7.0,1.0
342,M1750-E5-P55,55,2022-03-31,3,5,NaN,Natália Vieira,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,7.0,1.0
343,M1750-E5-P55,55,2022-03-31,3,5,NaN,Wilton Ladeira,NaN,NaN,7.0,1.0,NaN,NaN,NaN,NaN,NaN,7.0,1.0
395,M1928-E6-P57,57,2022-03-31,3,6,Brachiaria brizantha,Sabrina Beda,36.6,4.0,2.0,1.0,3.0,3.5,3.0,3.5,3.5,NaN,3.5
402,M1928-E6-P60,60,2022-03-31,3,6,Andropogon gayanus,Nivaldo Junior,44.8,4.5,2.5,1.5,2.5,4.0,3.0,3.0,4.0,NaN,1.5
615,M2109-E5-P133,133,2022-04-01,3,5,NaN,Lucas Tales,NaN,NaN,7.0,1.0,6.0,NaN,1.0,NaN,NaN,7.0,1.0
616,M2109-E5-P133,133,2022-04-01,3,5,NaN,Sabrina Beda,NaN,NaN,7.0,1.0,6.5,NaN,NaN,NaN,NaN,7.0,1.0


Note que os valores faltantes aparecem para algumas colunas e não para outras considerando uma mesma linha, de modo que, apesar de não termos informações para uma coluna, temos para outras.

Nesse caso, manterei os dados como estão, pois esses valores faltantes podem apresentar poder preditivo mais à frente. Vou salvar essa versão dos dados para eu não ter que repetir todo esse processo que fizemos de novo:

In [20]:
dados.to_excel("dados/dados_escores_processados.xlsx", index=False)